# **AI TECH INSTITUTE** · *Intermediate AI & Data Science*
### Classification Fundamentals: Encoding & Probabilities
**Instructor:** Amir Charkhi

### Learning Objectives
- Understand what classification is and how it differs from regression
- Learn how to handle categorical variables (one-hot encoding)
- Understand probability predictions in classification
- Interpret classification results

---

## 1. What is Classification?

**Classification** predicts which **category** or **class** something belongs to.

### Examples:
- 📧 **Email:** Spam or Not Spam
- 🏥 **Medical:** Disease or Healthy
- 💳 **Banking:** Fraud or Legitimate
- 🎓 **Education:** Pass or Fail
- 🌦️ **Weather:** Sunny, Rainy, or Cloudy

### Key Difference from Regression:

| Regression | Classification |
|------------|----------------|
| Predicts **numbers** | Predicts **categories** |
| "How much?" | "Which one?" |
| House price: $350,000 | Email: Spam or Not Spam |
| Temperature: 72°F | Weather: Sunny, Rainy, Cloudy |
| Sales: $5,430 | Customer: Will Buy or Won't Buy |

---

## 2. Our Simple Problem: Student Course Pass/Fail

Let's predict if a student will **pass** or **fail** a course based on:
- Study hours per week
- Attendance percentage
- Previous GPA
- Study method (Online, In-Person, Hybrid)

This is a **binary classification** problem: 2 classes (Pass or Fail)

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (8, 5)

print("✅ Libraries imported!")

---
## 3. Create a Simple Dataset

Let's create a small, easy-to-understand dataset with 20 students.

In [ ]:
# Create a simple dataset
data = {
    'StudentID': range(1, 21),
    'StudyHours': [15, 8, 12, 5, 18, 10, 7, 14, 20, 6, 
                   16, 9, 11, 4, 19, 13, 8, 17, 12, 5],
    'Attendance': [95, 70, 85, 60, 98, 75, 65, 90, 100, 68,
                   92, 72, 80, 55, 97, 88, 70, 95, 82, 58],
    'PreviousGPA': [3.8, 2.5, 3.2, 2.0, 3.9, 2.8, 2.3, 3.5, 4.0, 2.4,
                    3.7, 2.6, 3.0, 1.9, 3.8, 3.4, 2.5, 3.9, 3.1, 2.1],
    'StudyMethod': ['Online', 'In-Person', 'Hybrid', 'Online', 'Hybrid',
                    'In-Person', 'Online', 'Hybrid', 'Hybrid', 'In-Person',
                    'Online', 'In-Person', 'Hybrid', 'Online', 'Hybrid',
                    'Hybrid', 'In-Person', 'Online', 'Hybrid', 'Online'],
    'Passed': [1, 0, 1, 0, 1, 0, 0, 1, 1, 0,
               1, 0, 1, 0, 1, 1, 0, 1, 1, 0]  # 1 = Pass, 0 = Fail
}

df = pd.DataFrame(data)

print("📊 Student Course Performance Dataset")
print("="*60)
print(df.head(10))
print("\n📈 Dataset Info:")
print(f"   Total Students: {len(df)}")
print(f"   Passed: {df['Passed'].sum()}")
print(f"   Failed: {len(df) - df['Passed'].sum()}")

---
## 4. Understanding Our Data Types

We have two types of features:

### 📊 **Numerical Features** (already numbers)
- StudyHours: 5, 8, 12, 15, 18, 20...
- Attendance: 55, 60, 65, 70...
- PreviousGPA: 1.9, 2.0, 2.5, 3.0...

✅ **These are ready to use!** Machine learning models understand numbers.

### 📝 **Categorical Features** (text labels)
- StudyMethod: "Online", "In-Person", "Hybrid"

❌ **Problem:** Machine learning models can't understand text!

**Solution:** We need to convert text → numbers using **One-Hot Encoding**

In [ ]:
# Check data types
print("🔍 Data Types:")
print(df.dtypes)
print("\n💡 Notice: StudyMethod is 'object' (text), not a number!")

---
## 5. One-Hot Encoding: Converting Categories to Numbers

### ❌ **Bad Idea:** Just assign numbers
```
Online = 1
In-Person = 2  
Hybrid = 3
```
**Problem:** This implies Hybrid (3) > In-Person (2) > Online (1), which is NOT true!

### ✅ **Good Idea:** One-Hot Encoding

Create **separate binary columns** for each category:

| StudyMethod | → | StudyMethod_Online | StudyMethod_In-Person | StudyMethod_Hybrid |
|-------------|---|-------------------|----------------------|-------------------|
| Online      |   | 1                 | 0                    | 0                 |
| In-Person   |   | 0                 | 1                    | 0                 |
| Hybrid      |   | 0                 | 0                    | 1                 |

Each category gets its own column with:
- **1** = Yes, this is the category
- **0** = No, this is not the category

### 🎯 **Key Insight:** 
We can use **drop_first=True** to drop one column because if a student is NOT Online AND NOT In-Person, they MUST be Hybrid. This avoids redundancy.

In [ ]:
# BEFORE one-hot encoding
print("BEFORE One-Hot Encoding:")
print("="*60)
print(df[['StudentID', 'StudyMethod', 'Passed']].head())
print(f"\nShape: {df.shape}")

In [ ]:
# Apply one-hot encoding
df_encoded = pd.get_dummies(df, columns=['StudyMethod'], drop_first=True)

print("\nAFTER One-Hot Encoding:")
print("="*60)
print(df_encoded.head())
print(f"\nShape: {df_encoded.shape}")
print(f"\n💡 Notice: 'StudyMethod' is now 2 columns (In-Person and Online)")
print(f"   If both are 0, the student uses Hybrid method!")

In [ ]:
# Show the encoding more clearly
print("\n📊 How One-Hot Encoding Works:")
print("="*70)
comparison = df[['StudentID', 'StudyMethod']].copy()
comparison['Method_InPerson'] = df_encoded['StudyMethod_In-Person'].values
comparison['Method_Online'] = df_encoded['StudyMethod_Online'].values
print(comparison.head(10))
print("\n💡 Pattern:")
print("   In-Person → [1, 0]  (In-Person=1, Online=0)")
print("   Online    → [0, 1]  (In-Person=0, Online=1)")
print("   Hybrid    → [0, 0]  (In-Person=0, Online=0)")

---
## 6. Prepare Data for Machine Learning

Now we split our data into:
- **X** (Features): Everything except the target
- **y** (Target): What we want to predict (Passed)

In [ ]:
# Prepare features (X) and target (y)
X = df_encoded.drop(['StudentID', 'Passed'], axis=1)
y = df_encoded['Passed']

print("📊 Features (X):")
print(X.columns.tolist())
print(f"\nX shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"\n💡 We have {X.shape[1]} features to predict Pass/Fail")

In [ ]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print("✂️ Data Split:")
print(f"   Training: {len(X_train)} students ({len(X_train)/len(X)*100:.0f}%)")
print(f"   Testing:  {len(X_test)} students ({len(X_test)/len(X)*100:.0f}%)")
print(f"\n   Training - Passed: {y_train.sum()}, Failed: {len(y_train) - y_train.sum()}")
print(f"   Testing  - Passed: {y_test.sum()}, Failed: {len(y_test) - y_test.sum()}")

---
## 7. Train a Simple Classification Model

We'll use **Logistic Regression** - a simple but powerful classification algorithm.

### How it works:
1. Learns patterns from training data
2. For new students, calculates **probability** of passing
3. If probability > 0.5 → Predict "Pass"
4. If probability < 0.5 → Predict "Fail"

In [ ]:
# Train Logistic Regression model
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

print("✅ Model trained!")
print(f"\n📊 Model learned from {len(X_train)} students")

---
## 8. Understanding Predictions: Classes vs Probabilities

### Two Types of Predictions:

1. **Class Prediction (predict):**
   - Returns: 0 or 1 (Fail or Pass)
   - Based on threshold (default 0.5)

2. **Probability Prediction (predict_proba):**
   - Returns: Probability for each class
   - Shows **confidence** of prediction
   - Example: [0.2, 0.8] means 20% chance Fail, 80% chance Pass

In [ ]:
# Make predictions on test set
y_pred_class = model.predict(X_test)  # Predicted classes (0 or 1)
y_pred_proba = model.predict_proba(X_test)  # Predicted probabilities

print("🎯 Let's see predictions for first 5 test students:")
print("="*80)

results = pd.DataFrame({
    'Actual': y_test.values,
    'Predicted_Class': y_pred_class,
    'Prob_Fail': y_pred_proba[:, 0],
    'Prob_Pass': y_pred_proba[:, 1]
})

print(results.head())

print("\n💡 Understanding the output:")
print("   - Actual: What actually happened (0=Fail, 1=Pass)")
print("   - Predicted_Class: What model predicts (0=Fail, 1=Pass)")
print("   - Prob_Fail: Probability of failing (0-1)")
print("   - Prob_Pass: Probability of passing (0-1)")
print("   - These probabilities always add up to 1.0")

In [ ]:
# Detailed look at one prediction
print("\n🔍 Detailed Look at Student #0:")
print("="*80)

student_idx = 0
actual = y_test.values[student_idx]
pred_class = y_pred_class[student_idx]
prob_fail = y_pred_proba[student_idx, 0]
prob_pass = y_pred_proba[student_idx, 1]

print(f"Actual outcome:           {'PASSED' if actual == 1 else 'FAILED'}")
print(f"Predicted outcome:        {'PASSED' if pred_class == 1 else 'FAILED'}")
print(f"\nProbability of FAILING:   {prob_fail:.1%}")
print(f"Probability of PASSING:   {prob_pass:.1%}")
print(f"\nConfidence: {max(prob_fail, prob_pass):.1%}")

if pred_class == actual:
    print("\n✅ Prediction is CORRECT!")
else:
    print("\n❌ Prediction is WRONG!")

print(f"\n💡 How the decision was made:")
print(f"   Since Prob_Pass ({prob_pass:.3f}) {'>' if pred_class == 1 else '<'} 0.5")
print(f"   → Predict: {'PASS' if pred_class == 1 else 'FAIL'}")

---
## 9. Visualizing Probabilities

Let's visualize how confident the model is about each prediction.

In [ ]:
# Visualize probabilities
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Probability distribution
axes[0].hist(y_pred_proba[:, 1], bins=10, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].axvline(0.5, color='red', linestyle='--', linewidth=2, label='Decision Threshold')
axes[0].set_xlabel('Predicted Probability of PASSING', fontsize=11)
axes[0].set_ylabel('Number of Students', fontsize=11)
axes[0].set_title('Distribution of Predicted Probabilities', fontsize=12, pad=15)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Actual vs Predicted with confidence
correct = results['Actual'] == results['Predicted_Class']
colors = ['green' if c else 'red' for c in correct]
sizes = results['Prob_Pass'] * 200  # Size based on confidence

axes[1].scatter(range(len(results)), results['Prob_Pass'], 
                c=colors, s=sizes, alpha=0.6, edgecolors='black')
axes[1].axhline(0.5, color='red', linestyle='--', linewidth=2, label='Threshold=0.5')
axes[1].set_xlabel('Student Index', fontsize=11)
axes[1].set_ylabel('Predicted Probability of PASSING', fontsize=11)
axes[1].set_title('Prediction Confidence (Green=Correct, Red=Wrong)', fontsize=12, pad=15)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Insights:")
print("   - Larger circles = Higher confidence predictions")
print("   - Green = Correct predictions")
print("   - Red = Incorrect predictions")
print("   - Above 0.5 line = Predicted to PASS")
print("   - Below 0.5 line = Predicted to FAIL")

---
## 10. Model Performance

How well did our model do?

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_class)

print("📊 Model Performance:")
print("="*60)
print(f"Accuracy: {accuracy:.1%}")
print(f"\nOut of {len(y_test)} students:")
print(f"   ✅ Correctly predicted: {(y_test == y_pred_class).sum()}")
print(f"   ❌ Incorrectly predicted: {(y_test != y_pred_class).sum()}")

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_class)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Predicted FAIL', 'Predicted PASS'],
            yticklabels=['Actually FAILED', 'Actually PASSED'])
plt.title('Confusion Matrix: What Did We Get Right?', fontsize=13, pad=20)
plt.ylabel('Actual Outcome', fontsize=11)
plt.xlabel('Predicted Outcome', fontsize=11)
plt.tight_layout()
plt.show()

print("\n📊 Confusion Matrix Breakdown:")
print("="*60)
print(f"Top-Left (Correct):    {cm[0,0]} students correctly predicted to FAIL")
print(f"Bottom-Right (Correct): {cm[1,1]} students correctly predicted to PASS")
print(f"Top-Right (Wrong):     {cm[0,1]} students predicted PASS but actually FAILED")
print(f"Bottom-Left (Wrong):   {cm[1,0]} students predicted FAIL but actually PASSED")

---
## 11. Making Predictions for New Students

Let's predict for 2 new students we've never seen before!

In [ ]:
# Create new student data
new_students = pd.DataFrame({
    'StudyHours': [18, 6],
    'Attendance': [95, 60],
    'PreviousGPA': [3.7, 2.2],
    'StudyMethod': ['Hybrid', 'Online']
})

print("🆕 New Students:")
print("="*60)
print(new_students)

# Apply same encoding
new_students_encoded = pd.get_dummies(new_students, columns=['StudyMethod'], drop_first=True)

# Ensure same columns as training data
for col in X.columns:
    if col not in new_students_encoded.columns:
        new_students_encoded[col] = 0
        
new_students_encoded = new_students_encoded[X.columns]

print("\n✅ After encoding (ready for model):")
print(new_students_encoded)

In [ ]:
# Make predictions
new_pred_class = model.predict(new_students_encoded)
new_pred_proba = model.predict_proba(new_students_encoded)

print("\n🎯 PREDICTIONS FOR NEW STUDENTS:")
print("="*80)

for i in range(len(new_students)):
    print(f"\n👤 Student {i+1}:")
    print(f"   Study Hours: {new_students.iloc[i]['StudyHours']}")
    print(f"   Attendance:  {new_students.iloc[i]['Attendance']}%")
    print(f"   Previous GPA: {new_students.iloc[i]['PreviousGPA']}")
    print(f"   Study Method: {new_students.iloc[i]['StudyMethod']}")
    print(f"\n   📊 PREDICTION: {'WILL PASS ✅' if new_pred_class[i] == 1 else 'WILL FAIL ❌'}")
    print(f"   Confidence:")
    print(f"      Probability of FAILING: {new_pred_proba[i, 0]:.1%}")
    print(f"      Probability of PASSING: {new_pred_proba[i, 1]:.1%}")
    print("   " + "-"*60)

---
## 12. Key Takeaways

### ✅ What We Learned:

**1. Classification vs Regression:**
- Classification predicts **categories** (Pass/Fail, Yes/No)
- Regression predicts **numbers** (prices, temperatures)

**2. Handling Categorical Variables:**
- Can't feed text directly to ML models
- Use **one-hot encoding** to convert categories → binary columns
- Each category gets its own column with 1s and 0s
- `drop_first=True` removes redundancy

**3. Two Types of Predictions:**
- **predict()**: Returns class (0 or 1)
- **predict_proba()**: Returns probabilities [P(class 0), P(class 1)]
- Probabilities show **confidence** of prediction

**4. Decision Threshold:**
- Default threshold = 0.5
- If P(Pass) > 0.5 → Predict Pass
- If P(Pass) < 0.5 → Predict Fail
- This threshold can be adjusted based on business needs!

**5. Model Interpretation:**
- Confusion matrix shows where model succeeds/fails
- High probability = High confidence
- Low probability (near 0.5) = Model is uncertain

---

### 🎯 Next Steps:

Now that you understand the fundamentals, you're ready to:
1. **Learn different classification algorithms** (SVM, KNN, Trees, etc.)
2. **Compare model performance** using metrics beyond accuracy
3. **Optimize thresholds** for business goals
4. **Handle imbalanced datasets**

---

### 💡 Remember:

```python
# The classification workflow:
1. Load data
2. Encode categorical variables (one-hot encoding)
3. Split: X (features) and y (target)
4. Train model: model.fit(X_train, y_train)
5. Predict classes: model.predict(X_test)
6. Get probabilities: model.predict_proba(X_test)
7. Evaluate: accuracy, confusion matrix, etc.
```

---

**AI Tech Institute** | *Building Tomorrow's AI Engineers Today*